In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# Extract text from inside

def preprocess(folder):
    no_of_candidates = []
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    entailed_fragments = []
    paragraphs = []
    list_of_files = sorted(os.listdir(current_dir))
    if folder == 'task2_train':
        x=1
    else:
        x = 2
    for i in range(len(list_of_files)-x):
        if i%100 == 0:
            print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        g = open(current_dir+'/'+list_of_files[i]+"/entailed_fragment.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/paragraphs"))
        no_of_candidates.append(len(paragraph_filenames))
        a = f.read()
        b = g.read()
        base_cases.append(a)
        entailed_fragments.append(b)
        f.close()
        g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/paragraphs/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras,no_of_candidates
    
#train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras = preprocess('task2_train')
    
test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras,no_of_candidates = preprocess('task2_test')

Processing  0


In [7]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs
test_entailed_fragments = filt(test_entailed_fragments)
test_paragraphs = filt(test_paragraphs)
test_base_cases = filt(test_base_cases)

In [4]:
data = test_entailed_fragments + test_paragraphs
documents = test_entailed_fragments + test_paragraphs + test_base_cases
def gen_labels(data):
    labels = []
    for i in range(len(test_entailed_fragments)):
        labels.append("base_case_"+str(i+1))
    for i in range(len(test_entailed_fragments)):
        for j in range(no_of_candidates[i]):
            labels.append("candidate_"+str(i+1)+"_"+str(j+1))
    for i in range(len(test_entailed_fragments)):
        labels.append("extra_"+str(i+1))
    return labels
labels = gen_labels(documents)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from gensim.summarization.bm25 import get_bm25_weights
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        #print(i)
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs


In [10]:
from gensim.summarization.bm25 import get_bm25_weights
lst2 = test_entailed_fragments + test_paragraphs + test_base_cases
lst3 = filt(lst2)
arr = []
for elem in lst3:
    arr.append(elem.split())
corpus = arr
result = get_bm25_weights(corpus, n_jobs=-1)

In [6]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model")

2019-03-30 20:18:13,669 : INFO : loading Doc2Vec object from doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model
2019-03-30 20:18:13,789 : INFO : loading vocabulary recursively from doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model.vocabulary.* with mmap=None
2019-03-30 20:18:13,790 : INFO : loading trainables recursively from doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model.trainables.* with mmap=None
2019-03-30 20:18:13,790 : INFO : loading wv recursively from doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model.wv.* with mmap=None
2019-03-30 20:18:13,791 : INFO : loading docvecs recursively from doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model.docvecs.* with mmap=None
2019-03-30 20:18:13,792 : INFO : loaded doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model


In [27]:
for i in range(61):
    
    lst = []
    for j in range(200):
        a = model.docvecs.similarity(labels[i],"candidate_"+train_list_of_files[i]+"_"+str(j+1))
        b = result[0][j+1]
        lst.append((a*b,j+1))
    lst.sort(reverse=True)
    cutoff = ((lst[0][0] + lst[1][0])/2)*0.80
    for j in range(10):
        if lst[j][0] > cutoff:
            print("t1-"+str(i+1),lst[j][1],"IITPdocBM")

t1-1 48 IITPdocBM
t1-2 154 IITPdocBM
t1-3 180 IITPdocBM
t1-3 124 IITPdocBM
t1-4 137 IITPdocBM
t1-4 182 IITPdocBM
t1-4 65 IITPdocBM
t1-5 174 IITPdocBM
t1-5 163 IITPdocBM
t1-5 130 IITPdocBM
t1-5 110 IITPdocBM
t1-5 111 IITPdocBM
t1-6 124 IITPdocBM
t1-6 62 IITPdocBM
t1-6 85 IITPdocBM
t1-6 168 IITPdocBM
t1-6 187 IITPdocBM
t1-6 184 IITPdocBM
t1-6 45 IITPdocBM
t1-6 167 IITPdocBM
t1-6 58 IITPdocBM
t1-6 70 IITPdocBM
t1-7 178 IITPdocBM
t1-7 22 IITPdocBM
t1-8 88 IITPdocBM
t1-8 157 IITPdocBM
t1-8 182 IITPdocBM
t1-9 71 IITPdocBM
t1-10 180 IITPdocBM
t1-10 187 IITPdocBM
t1-10 51 IITPdocBM
t1-10 154 IITPdocBM
t1-11 11 IITPdocBM
t1-11 111 IITPdocBM
t1-11 25 IITPdocBM
t1-11 105 IITPdocBM
t1-11 2 IITPdocBM
t1-11 49 IITPdocBM
t1-11 195 IITPdocBM
t1-12 7 IITPdocBM
t1-12 124 IITPdocBM
t1-12 50 IITPdocBM
t1-12 194 IITPdocBM
t1-12 11 IITPdocBM
t1-12 192 IITPdocBM
t1-12 52 IITPdocBM
t1-13 95 IITPdocBM
t1-14 156 IITPdocBM
t1-14 79 IITPdocBM
t1-15 18 IITPdocBM
t1-16 124 IITPdocBM
t1-16 58 IITPdocBM
t1-17 127 IIT

In [20]:
for i in range(44):
    #print("\nPredicting  For ",i)
    lst = []
    for j in range(no_of_candidates[i]):
        #lst.append((model.docvecs.similarity(labels[i],labels[285+i*200+j]),j+1))
        a = result[i][44+sum(no_of_candidates[:i])+j]
        b = model.docvecs.similarity(labels[i],"candidate_"+str(i+1)+"_"+str(j+1))
        lst.append((a*b,j+1))
        #print(labels[44+sum(no_of_candidates[:i])+j])
    lst.sort(reverse=True)
    cutoff = ((lst[0][0] + lst[1][0])/2)*0.90
    for j in range(1):
        if lst[j][0] > cutoff:
            print("t2-"+str(i+1),lst[j][1],"IITP2BM25")

t2-1 1 IITP2BM25
t2-2 12 IITP2BM25
t2-3 18 IITP2BM25
t2-4 15 IITP2BM25
t2-5 59 IITP2BM25
t2-6 25 IITP2BM25
t2-7 27 IITP2BM25
t2-8 7 IITP2BM25
t2-9 14 IITP2BM25
t2-10 5 IITP2BM25
t2-11 24 IITP2BM25
t2-12 12 IITP2BM25
t2-13 20 IITP2BM25
t2-14 13 IITP2BM25
t2-15 42 IITP2BM25
t2-16 7 IITP2BM25
t2-17 31 IITP2BM25
t2-18 13 IITP2BM25
t2-19 7 IITP2BM25
t2-20 26 IITP2BM25
t2-21 58 IITP2BM25
t2-22 24 IITP2BM25
t2-23 16 IITP2BM25
t2-24 16 IITP2BM25
t2-25 47 IITP2BM25
t2-26 17 IITP2BM25
t2-27 16 IITP2BM25
t2-28 21 IITP2BM25
t2-29 34 IITP2BM25
t2-30 29 IITP2BM25
t2-31 24 IITP2BM25
t2-32 24 IITP2BM25
t2-33 8 IITP2BM25
t2-34 26 IITP2BM25
t2-35 7 IITP2BM25
t2-36 18 IITP2BM25
t2-37 12 IITP2BM25
t2-38 13 IITP2BM25
t2-39 12 IITP2BM25
t2-40 31 IITP2BM25
t2-41 15 IITP2BM25
t2-42 20 IITP2BM25
t2-43 16 IITP2BM25
t2-44 25 IITP2BM25


In [11]:
for j in range(200):
    lst.append((result[0][j+1],j+1))
lst.sort(reverse=True)
cutoff = ((lst[0][0] + lst[1][0])/2)*0.90
for j in range(10):
    if lst[j][0] > cutoff:
        print("t1-"+str(i+1),lst[j][1],"IITPBM25")

In [17]:
result[0][0]

-75020.47405382311

In [ ]:
325,1486,855

In [ ]:
588/2421

In [ ]:
468/1486

In [ ]:
import re
a = re.search("Baban","I am Baban Gain Baban GAin")
print(a)